In [5]:
! python train_unetKD.py   --root_path data/Synapse   --output_dir runs/evit_unet   --batch_size 8   --max_epochs 10   --img_size 224 --val_list lists/lists_Synapse/val.txt

/home/hxy/miniconda3/envs/evit-unet/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/hxy/miniconda3/envs/evit-unet/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
Namespace(root_path='data/Synapse/train_npz', dataset='Synapse', list_dir='./lists/lists_Synapse', num_classes=9, output_dir='runs/evit_unet', max_iterations=30000, max_epochs=10, batch_size=8, n_gpu=1, deterministic=1, base_lr=0.002, img_size=224, seed=1234, opts=None, zip=False, cache_mode='part', resume=None, accumulation_steps=None, use_checkpoint=False, amp_opt_level='O1', tag=None, eval=False, thr

In [ ]:
! python train_unetKD.py   --root_path data/Synapse   --output_dir runs/evit_unet   --batch_size 8   --max_epochs 101   --img_size 224 --resume runs/evit_unet/epoch_68.pth

In [ ]:
! python test_diy.py \
  --model_path runs/evit_unet/epoch_20.pth \
  --test_dir data/Synapse/test_vol_h5 \
  --save_pred predictions \
  --ref_dir datasets/RawData/Training/label \
  --img_size 224 \
  --num_classes 9


In [ ]:
! python test_fast.py \
  --model_path runs/evit_unet/epoch_68.pth \
  --test_dir data/Synapse/test_vol_h5 \
  --save_pred predictions/fast \
  --ref_dir datasets/RawData/Training/label \
  --img_size 224 \
  --num_classes 9 \
  --batch_size 256


In [ ]:
import nibabel as nib
import numpy as np

seg = nib.load('datasets/RawData/Training/label/label0001.nii.gz')
# seg = nib.load('datasets/RawData/Training/img/img0001.nii.gz')
label_data = seg.get_fdata()
print("Unique classes in .nii.gz:", np.unique(label_data))


In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# 加载 .nii.gz 文件（标签）
seg_path = 'datasets/RawData/Training/label/label0037.nii.gz'
seg_nifti = nib.load(seg_path)
seg_data = seg_nifti.get_fdata().astype(np.uint8)

# 打印标签种类
print("Unique labels:", np.unique(seg_data))

# 显示中间某一张切片（沿 z 轴）
slice_idx = 9
print("Slice index:", slice_idx)
plt.figure(figsize=(8, 6))
plt.imshow(seg_data[:, :, slice_idx], cmap='nipy_spectral')
plt.title(f"Label Slice {slice_idx}")
plt.colorbar()
plt.show()


In [ ]:
import numpy as np
import SimpleITK as sitk

# 路径
pred_path = "predictions/img0001_pred.nii.gz"
ref_path = "datasets/RawData/Training/label/label0001.nii.gz"
output_path = "predictions/label0001_pred_fixed.nii.gz"

# 读取预测与参考图像
pred_img = sitk.ReadImage(pred_path)
ref_img = sitk.ReadImage(ref_path)

# 转为 numpy 数组
pred_array = sitk.GetArrayFromImage(pred_img)  # [D, H, W]

# === 方向修正逻辑 ===
# Step 1: 对每一层做旋转（顺时针90°）+ 水平反转
for i in range(pred_array.shape[0]):
    pred_array[i] = np.fliplr(np.rot90(pred_array[i], k=3))

# # Step 2: 修正层顺序（Z 轴翻转）
# pred_array = pred_array[::-1, :, :]

# === 保存为 NIfTI 并复用标签的 header ===
fixed_img = sitk.GetImageFromArray(pred_array.astype(np.uint8))
fixed_img.SetSpacing(ref_img.GetSpacing())
fixed_img.SetOrigin(ref_img.GetOrigin())
fixed_img.SetDirection(ref_img.GetDirection())

sitk.WriteImage(fixed_img, output_path)
print("✅ 修正完成并保存:", output_path)